In [1]:
%run ts_processing.py
%run ts_modelling.py

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns 

import inspect
import warnings
warnings.filterwarnings("ignore")

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
plt.style.use('seaborn-colorblind')
sns.set_theme()
%matplotlib inline
plt.rcParams["figure.figsize"] = (20, 10)
plt.rcParams['figure.dpi'] = 150

# Functions

In [3]:
print(inspect.getsource(data_processing)) # multivariate is the same without the datetime features

def data_processing(url_base, univ=True):
    """
    Takes NordPool's data in DataFrame format and corrects the misnaming of the hours,
    Assigns the correct index,
    Changes the price units to €/KWh (from €/MWh),
    Normalizes the prices,
    Adds datetime features to help explicitly infer the seasonality.
    """
    
    # Get data into a DataFrame
    currency = 'eur'
    filetype = ".xls"
    years = ["17", "18", "19", "20", "21", "22"] #Years we want to get historical data
    for y in years:
        if y == years[0]: prices = read_spot_prices(url_base+currency+str(y)+filetype)
        else: 
            df2 = read_spot_prices(url_base+currency+str(y)+filetype)
            prices = prices.append(df2)
    prices.dropna(subset = ["date"], how = "any", axis = 0, inplace=True)
    prices.drop_duplicates(subset=["date"], keep = "last", ignore_index = True, inplace=True)
    price_dat = prices.drop("date", axis = 1)
    dat = []
    # Solve the misnaming of Nordpool (24h to 00h f

In [4]:
regions = {'NO1':'os-',
           'NO3':'trh-',
           'NO5':'ber-',
           'SE1':'lul',
           'SE3':'sto',
           'SE4':'mal',
           'DK1':'ode',
           'DK2':'cph'}

# Univariate

In [5]:
def data_processing(url_base, univ=True):
    """
    Takes NordPool's data in DataFrame format and corrects the misnaming of the hours,
    Assigns the correct index,
    Changes the price units to €/KWh (from €/MWh),
    Normalizes the prices,
    Adds datetime features to help explicitly infer the seasonality.
    """
    
    # Get data into a DataFrame
    currency = 'eur'
    filetype = ".xls"
    years = ["17", "18", "19", "20", "21", "22"] #Years we want to get historical data
    for y in years:
        if y == years[0]: prices = read_spot_prices(url_base+currency+str(y)+filetype)
        else: 
            df2 = read_spot_prices(url_base+currency+str(y)+filetype)
            prices = prices.append(df2)
    prices.dropna(subset = ["date"], how = "any", axis = 0, inplace=True)
    prices.drop_duplicates(subset=["date"], keep = "last", ignore_index = True, inplace=True)
    price_dat = prices.drop("date", axis = 1)
    dat = []
    # Solve the misnaming of Nordpool (24h to 00h format)
    for ii, ro in price_dat.iterrows():
        dat.extend(ro.values)
    
    # Remove first 23 hours since day 1st jan 2017 is ordered incorrectly
    dat = dat[23:] 
    # Keep the values for the dates that have passed by, remove future prices
    dat = dat[:45505] 
    
    # Give DataFrame format with date range
    date_rng = pd.date_range(start='1/02/2017', end='3/13/2022', freq='H')
    df = pd.DataFrame(data = dat, columns = ["price"])
    
    # Add datetime features
    df["datetime"] = date_rng
    df = df[:-1]
    df.interpolate(inplace = True) # To check if ok use """df[1990:2000]"""
    
    if univ == True:
        df["weekday"] = df["datetime"].dt.weekday
        df["week"] = df["datetime"].dt.week
        df["day"] = df["datetime"].dt.day
    
    return df


In [6]:
### Save univariate series to each regions folder
for reg in regions.keys():
    url_base=f'NordPool/{reg}/{regions[reg]}'
    df = data_processing(url_base)
    df.to_csv(f'NordPool/{reg}/Spot_price_{reg}.csv')

# Multivariate

In [15]:
### Save multivariate data to NordPool's folder
reg_num = 0
for reg in regions.keys():
    reg_num += 1
    url_base=f'NordPool/{reg}/{regions[reg]}'
    if reg_num==1:
        df = data_processing(url_base, univ=False)
    else:
        df2 = data_processing(url_base, univ=False)
        df=df.merge(df2, on='datetime', how = 'outer', suffixes=(None,f'_{reg}'))
df.interpolate(inplace = True)
df.columns=['NO1', 'datetime', 'NO3', 'NO5', 'SE1', 'SE3', 'SE4', 'DK1', 'DK2']
df["datetime"] = pd.to_datetime(df["datetime"])
df["weekday"] = df["datetime"].dt.weekday
df["week"] = df["datetime"].dt.week
df["day"] = df["datetime"].dt.day
df["hour"] = df["datetime"].dt.hour
df.to_csv("NordPool/Spot_Price_Nordics.csv")

In [16]:
df.sample(15)

,NO1,datetime,NO3,NO5,SE1,SE3,SE4,DK1,DK2,weekday,week,day,hour
30765,1.30,2020-07-06 21:00:00,1.49,1.30,4.87,4.87,4.87,4.87,4.87,0,28,6,21
33188,20.82,2020-10-15 20:00:00,17.71,20.33,28.71,54.36,54.36,54.36,54.36,3,42,15,20
14707,59.74,2018-09-06 19:00:00,68.95,59.74,68.95,68.95,68.95,73.89,73.89,3,36,6,19
20218,41.97,2019-04-24 10:00:00,41.97,41.97,41.97,41.97,41.97,41.97,41.97,2,17,24,10
24741,40.57,2019-10-29 21:00:00,40.57,40.57,40.57,40.57,53.91,53.91,53.91,1,44,29,21
39720,59.80,2021-07-15 00:00:00,51.10,59.80,59.80,59.80,59.80,75.68,75.68,3,28,15,0
22583,39.87,2019-07-31 23:00:00,41.02,39.87,42.48,42.48,42.84,54.51,54.51,2,31,31,23
22814,35.11,2019-08-10 14:00:00,35.11,35.11,35.11,35.11,35.11,-16.04,35.11,5,32,10,14
8916,35.87,2018-01-08 12:00:00,33.15,35.87,33.15,35.87,41.54,41.54,41.54,0,2,8,12
6395,30.83,2017-09-25 11:00:00,35.89,30.83,35.89,35.89,35.89,35.89,51.10,0,39,25,11
